## Ising Model GAN 

In [1]:
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import sys
sys.path.insert(0, "..")
import time
from IPython import display
from model import make_discriminator_model, make_generator_model, train_step
from pathlib import Path
data_path = Path("../../GetData/Python/Data")

2024-02-16 10:07:37.214049: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-16 10:07:37.252958: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-16 10:07:37.252988: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-16 10:07:37.253927: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-16 10:07:37.259767: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-16 10:07:37.260443: I tensorflow/core/platform/cpu_feature_guard.cc:1

## Load data

In [2]:
import tensorflow as tf
from input_pipeline import dataset_tfrecord_pipeline

class DataIterator:
    def __init__(self, datasets):
        self.datasets = datasets
    
    def __iter__(self):
        self.data_iterators = [iter(data) for data in self.datasets]
        return self
    
    def __next__(self):
        data_list = []
        temp_list = []
        for index, data_iterator in enumerate(self.data_iterators):
            data = next(data_iterator)
            data_list.append(data)
            temp = np.zeros((data.shape[0], len(self.data_iterators)))
            temp[:, index] = 1
            temp_list.append(temp)
        temps = tf.concat(temp_list, axis=0)
        data = tf.concat(data_list, axis=0)
        return data, temps
        
def cvae_dataset(data_dir, temps, batch_size=100, flatten=False):
    if isinstance(data_dir, str):
        data_dir = Path(data_dir)
    
    assert batch_size % len(temps) == 0, "Batch size must be divisible by the number of temperatures"

    datasets = []
    for temp in temps:
        dataset = dataset_tfrecord_pipeline(data_dir / f"Data{temp:.1f}.tfrecord", flatten=flatten, batch_size=batch_size // len(temps))
        datasets.append(dataset)
    gen = DataIterator(datasets)
    dataset = tf.data.Dataset.from_generator(lambda: gen, output_signature = (tf.TensorSpec(shape=(None, 32, 32, 1), dtype=tf.float32), tf.TensorSpec(shape=(None, len(temps)), dtype=tf.float32)))
    return dataset

In [3]:
temps = [2.0, 2.5, 2.9]
batch_size = 120
train_dataset = cvae_dataset(data_path, temps, batch_size=batch_size, flatten=False)

In [4]:
from datetime import datetime
from pathlib import Path
# Get the current date and time
current_datetime = datetime.now()

# Format it to include date, hour, and minutes
formatted_datetime = current_datetime.strftime("%Y-%m-%d_%H-%M")

results = Path("results/" + formatted_datetime)
results.mkdir()

In [5]:
def generate_and_save_images(model, epoch, test_input):
  predictions = tf.round(model(test_input, training=False))

  fig = plt.figure(figsize=(4,4))
  
  for i in range(predictions.shape[0]):
      plt.subplot(4, 4, i+1)
      plt.imshow(predictions[i, :, :, 0])
      plt.axis('off')
        
  plt.savefig(results / 'image_at_epoch_{:04d}.png'.format(epoch))
  plt.show()

def train(dataset, epochs, batch_size, gen_loss_log, disc_loss_log):  
  for epoch in range(epochs):
    start = time.time()

    for images in tqdm(dataset): train_step(images, gen_loss_log, disc_loss_log, batch_size, noise_dim, generator, discriminator, generator_optimizer, discriminator_optimizer)

    display.clear_output(wait=True) 
    generate_and_save_images(
      generator,
      epoch + 1,
      random_vector_for_generation
    ) 
    print (f"Time taken for epoch {epoch} is {time.time()- start} sec")
    

noise_dim = 100
num_examples_to_generate = 16
random_vector_for_generation = tf.random.normal([num_examples_to_generate,
                                                 noise_dim])
generator = make_generator_model(len(temps))
discriminator = make_discriminator_model()
generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)

# Train

In [6]:
EPOCHS=2
gen_loss_log=[]
disc_loss_log=[]

In [8]:
%%time
train(train_dataset, EPOCHS, batch_size, gen_loss_log, disc_loss_log)

13it [00:18,  1.44s/it]
Traceback (most recent call last):
  File "/home/ps/.local/lib/python3.11/site-packages/IPython/core/magics/execution.py", line 1332, in time
    out = eval(code, glob, local_ns)
          ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "<timed eval>", line 1, in <module>
  File "/tmp/ipykernel_102682/2590414623.py", line 18, in train
    for images in tqdm(dataset): train_step(images, gen_loss_log, disc_loss_log, batch_size, noise_dim, generator, discriminator, generator_optimizer, discriminator_optimizer)
                                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/ps/Code/FUW/Ising/Model/GAN/model.py", line 74, in train_step
    generated_images = generator(noise_t, training=True)
                       ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/ps/.local/lib/python3.11/site-packages/keras/src/utils/traceback_utils.py", line 65, in error_han

Unexpected exception formatting exception. Falling back to standard exception


In [ ]:
noise = tf.random.normal([batch_size, noise_dim])
images, temps = data
noise_t = tf.concat([noise, temps], axis=1)

## Plot the loss of the generator and discriminator

In [ ]:
fig, ax1 = plt.subplots()

ax1.plot(np.asarray(disc_loss_log), color='tab:red')
ax1.set_xlabel('Epochs')
ax1.set_ylabel('Discriminator Loss', color='tab:red')
ax1.tick_params(axis='y', labelcolor='tab:red')

ax2 = ax1.twinx()
ax2.plot(np.asarray(gen_loss_log), color='tab:blue')
ax2.set_ylabel('Generator Loss', color='tab:blue')
ax2.tick_params(axis='y', labelcolor='tab:blue')

plt.show()

In [ ]:
generator.load_weights('generator_model.keras')

In [ ]:
plt.figure(figsize=(10, 10))
predictions = generator(random_vector_for_generation, training=False)
for i in range(16):
    plt.subplot(4, 4, i+1)
    spins = np.random.binomial(1, predictions[i, :, :])
    plt.imshow(spins)
    plt.axis('off')
plt.show()

In [ ]:
print(predictions.shape)